In [1]:
import pyspark.sql.functions as f

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7569,application_1589299642358_2064,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure
{"conf": {
    "spark.app.name": "datavirus_final"
}}

A session has already been started. If you intend to recreate the session with new configurations, please include the -f argument.


In [3]:
# Initialization
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7563,application_1589299642358_2058,pyspark,idle,Link,Link,
7565,application_1589299642358_2060,pyspark,idle,Link,Link,
7567,application_1589299642358_2062,pyspark,idle,Link,Link,
7568,application_1589299642358_2063,pyspark,busy,Link,Link,
7569,application_1589299642358_2064,pyspark,idle,Link,Link,✔
7570,application_1589299642358_2065,pyspark,busy,Link,Link,


In [11]:
sbb = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
sbb.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
|betriebstag|   fahrt_bezeichner|betreiber_id|betreiber_abk|betreiber_name|produkt_id|linien_id|linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|  bpuic|haltestellen_name|ankunftszeit|an_prognose|an_prognose_status|    abfahrtszeit|ab_prognose|ab_prognose_status|durchfahrt_tf|
+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
| 03.09.2018|80:06____:17010:000|   80:06____|           DB|   DB Regio AG|       Zug|    17010|    

In [13]:
sbb = (
    sbb
    
    # filter out any trips that we are not interested in
    .where(sbb.zusatzfahrt_tf == 'false')
    .where(sbb.faellt_aus_tf == 'false')
    .where(sbb.durchfahrt_tf == 'false')
    .where(sbb.ankunftszeit.isNotNull())
    .where(sbb.an_prognose.isNotNull())
    .where(sbb.an_prognose_status.isin('REAL', 'PROGNOSE'))
    
    # calculate delay
    .withColumn('ankunftszeit_ts', f.unix_timestamp(f.col('ankunftszeit'), "dd.MM.yyyy HH:mm").cast('long'))
    .withColumn('an_prognose_ts', f.unix_timestamp(f.col('an_prognose'), "dd.MM.yyyy HH:mm:ss").cast('long'))
    .withColumn('delay', f.col('an_prognose_ts') - f.col('ankunftszeit_ts'))
    
    # sometimes there are more multiple REAL and PROGNOSE values for the exactly same trip on the same day
    # and time. There is no way which one is the closest to the actual arrival time of the train so we 
    # just drop all duplicates and keep one "randomly".
    .select(['betriebstag', 'fahrt_bezeichner', 'bpuic', 'ankunftszeit', 'an_prognose_status', 'delay'])
    .dropDuplicates(['betriebstag', 'fahrt_bezeichner', 'bpuic', 'ankunftszeit', 'an_prognose_status'])
).persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
delays = (
    sbb
    
    # for each trip we only kept one REAL and one PROGNOSE row
    # now we aggregate them into the same row for each trip
    .groupBy(['betriebstag', 'fahrt_bezeichner', 'bpuic', 'ankunftszeit'])
    .agg(
        f.first(f.when(sbb.an_prognose_status == 'REAL', sbb.delay).otherwise(None)).alias('real_delay'),
        f.first(f.when(sbb.an_prognose_status == 'PROGNOSE', sbb.delay).otherwise(None)).alias('prognose_delay'),
    )
    
    # if there is REAL delay we select it, otherwise we take the PROGNOSE delay 
    .withColumn('delay', f.when(f.col('real_delay').isNotNull(), f.col('real_delay')).otherwise(f.col('prognose_delay')))
    .withColumn('is_delayed', f.when(f.col('delay') > 0, 1).otherwise(0))
)

delays.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+-------+----------------+----------+--------------+-----+----------+
|betriebstag|    fahrt_bezeichner|  bpuic|    ankunftszeit|real_delay|prognose_delay|delay|is_delayed|
+-----------+--------------------+-------+----------------+----------+--------------+-----+----------+
| 01.01.2018|85:146:29487-01177-1|8576853|01.01.2018 11:47|      null|            44|   44|         1|
| 01.01.2018|85:146:30274-01177-1|8576594|01.01.2018 18:37|      null|           120|  120|         1|
| 01.01.2018|85:3849:77145-050...|8576197|01.01.2018 13:08|      null|            43|   43|         1|
| 01.01.2018|   85:801:35178-3606|8508749|01.01.2018 16:10|      null|             0|    0|         0|
| 01.01.2018|85:807:68638-04151-1|8590486|01.01.2018 11:26|      null|             0|    0|         0|
| 01.01.2018|85:823:459478-034...|8588489|01.01.2018 11:49|      null|            82|   82|         1|
| 01.01.2018|85:827:459893-024...|8507061|01.01.2018 16:15|      null|   

In [25]:
probability = (
    delays
    
    # create an extra column that holds only time of the day without the date part
    .withColumn('ankunftszeit_time', delays.ankunftszeit.substr(11, 15)).show(5)
    
    # group identical trips over all days
    .groupBy(['fahrt_bezeichner', 'bpuic', 'ankunftszeit_time'])
    
    # calculate delay probability and 1/(mean delay) for each trip
    .agg(
        f.mean(delays.is_delayed).alias('delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('delay_parameter'),
        f.count(delays.delay).alias('n')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'NoneType' object has no attribute 'groupBy'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'groupBy'



In [26]:
probability.write.format("orc").save("/user/gflueck/delay_analysis.orc", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o423.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:224)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperat

In [27]:
probability = spark.read.orc("/user/gflueck/delay_analysis.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
u'Unable to infer schema for ORC. It must be specified manually.;'
Traceback (most recent call last):
  File "/hdata/sdg/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2064/container_e06_1589299642358_2064_01_000001/pyspark.zip/pyspark/sql/readwriter.py", line 476, in orc
    return self._df(self._jreader.orc(_to_seq(self._spark._sc, path)))
  File "/hdata/sdg/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2064/container_e06_1589299642358_2064_01_000001/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/hdata/sdg/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2064/container_e06_1589299642358_2064_01_000001/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u'Unable to infer schema for ORC. It must be specified manually.;'



In [28]:
probability.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o392.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 4 times, most recent failure: Lost task 0.3 in stage 24.0 (TID 2240, iccluster067.iccluster.epfl.ch, executor 35): java.io.FileNotFoundException: File does not exist: /user/gflueck/delay_analysis.orc/part-00145-00ad3400-2a55-45c7-aeb9-669d3b223744-c000.snappy.orc
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:153)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:1927)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:738)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.g

In [29]:
probability.select('n').describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o526.describe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 35 in stage 25.0 failed 4 times, most recent failure: Lost task 35.3 in stage 25.0 (TID 2319, iccluster066.iccluster.epfl.ch, executor 32): java.io.FileNotFoundException: File does not exist: /user/gflueck/delay_analysis.orc/part-00193-00ad3400-2a55-45c7-aeb9-669d3b223744-c000.snappy.orc
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:153)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:1927)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:738)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.g